- zs_dataset_list2を使用

In [1]:
# imports without torch
import numpy as np
import argparse
import os
import os.path
import visdom
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import json
import sys
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

##########
# imports torch
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
##########

# imports models
import model.segnet as segnet
import model.encoder as encoder  # label->word
import model.decoder as decoder  # word->label
import zs_dataset_list as datasets

# imports utility
import make_log as flog

###mapping function and GT_list###

In [2]:
tr_map_te=np.asarray([
26,
4,
4,
4,
4,
4,
4,
4,
4,
3,
3,
3,
3,
3,
3,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
1,
1,
1,
1,
1,
1,
1,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
10,
10,
10,
10,
10,
10,
10,
10,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
7,
7,
7,
7,
7,
7,
6,
6,
6,
6,
6,
6,
5,
5,
5,
5,
5,
5,
5,
5,
19,
19,
15,
17,
17,
15,
20,
16,
25,
22,
23,
23,
19,
19,
14,
20,
20,
19,
20,
12,
20,
16,
22,
22,
22,
22,
22,
15,
11,
18,
18,
20,
15,
12,
12,
13,
17,
15,
20,
19,
25,
20,
15,
13,
12,
19,
16,
25,
12,
19,
15,
25,
12,
12,
16,
12,
11,
12,
13,
17,
19,
18,
12,
11,
20,
14,
17,
12,
13,
20,
13,
15,
16,
20,
17,
19,
19,
15,
18,
24,
24,
24,
24,
24,
24,
24,
11,
11,
21,
21,
13,
26,
])
GT_list = [35, 26, 23, 9, 1, 83, 77, 72, 61, 51, 43, 154, 148,
           149, 105, 123, 112, 127, 152, 167, 109, 179, 116, 102,
           175, 99]

↓ define(1)

In [3]:
class p_args:
    load=False
    config="MSE_batch12_mask"
    test=False
    no_cuda=False

In [4]:
# set config
f_config = open(os.path.join("./config", p_args.config + ".json"), "r")
args = json.load(f_config)
for key, value in args.items():
    if value == "true":
        args[key] = True
    elif value == "false":
        args[key] = False
print(args)

# device settings
p_args.cuda = not p_args.no_cuda and torch.cuda.is_available()
USE_CUDA = p_args.cuda

{'model': True, 'encoder': False, 'decoder': False, 'input_nbr': 3, 'target_nbr': 100, 'semantic_nbr': 0, 'batch_size': 12, 'epochs': 10, 'lr': 0.0001, 'momentum': 0.1, 'seed': 1, 'lamda': 0, 'gamma': 1, 'input_root': './data/train/input', 'target_root': './data/train/zs_target', 'map_root': './data/train/target', 'HOG_root': './data/train/input_HOG', 'filenames': './data/train/zs_names.txt', 'semantic_filename': './v_class/class.txt', 'project_dir': './model/MSE_batch12_mask', 'model_load_pth': './model/MSE_batch12_mask/segnet.pth', 'head_load_pth': '', 'save_pth': 'segnet.pth', 'output_dir': './data/MSE_batch12_mask', 'n_components': 100, 'PCA': False, 'cos_similarity': False, 'jaccard_similarity': False, 'ZSL': True, 'SSE': False, 'MASK': True, 'HOG': False}


↓ define(2)

In [5]:
input_nbr = args["input_nbr"]  # 入力次元数
target_nbr = args["target_nbr"]  # 出力次元数

In [6]:
# set the seed
torch.manual_seed(args["seed"])
if p_args.cuda:
    torch.cuda.manual_seed(args["seed"])

↓ define(3)

In [7]:
model = segnet.SegNet(input_nbr, target_nbr, args["momentum"])

↓ define(4)

In [8]:
if args["model"] is True:
    if USE_CUDA:  # convert to cuda if needed
        model.cuda()
    else:
        model.float()
    model.eval()
    print(model)

SegNet(
  (conv11): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv21): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn21): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv22): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn22): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv31): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn31): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv32): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn32): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

↓ define(5)

In [9]:
# Create visdom
vis = visdom.Visdom()

# init window
if p_args.test is False:
    """model"""
    win_acc = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_accuracy',
            xlabel='epoch',
            ylabel='accuracy',
            width=800,
            height=400
        )
    )
    win_tacc = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='test_accuracy',
            xlabel='epoch',
            ylabel='accuracy',
            width=800,
            height=400
        )
    )
    win = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            # ytickmin=0.0,
            # ytickmax=1.0,
            width=800,
            height=400
        )
    )

In [10]:
# Create log model
f_log = flog.make_log(args["project_dir"])

↓ define(6)

In [11]:
optimizer = optim.Adam(model.parameters(), lr=args["lr"])

# Train Code

In [12]:
def model_train(epoch, trainloader):
    # set model to train mode
    model.train()

    # define a loss
    # 今回の場合背景クラスを考慮しないので重み付けはしない
    if USE_CUDA:
        loss = nn.MSELoss(size_average=True).cuda()
        l1_loss = nn.L1Loss(size_average=False).cuda()
    else:
        loss = nn.MSELoss(size_average=True)
        l1_loss = nn.L1Loss(size_average=False)

    # define total_loss
    total_loss = 0

    # define epoch_size
    epoch_size = len(trainloader)

    # define batch_loss
    batch_loss = 0

    # define lamda
    lamda = args["lamda"]

    # define annotations
    v_array = trainloader.dataset.v_array
    v_array = torch.from_numpy(v_array)
    if USE_CUDA:
        v_array = v_array.cuda()
    
    # define tqdm message
    t = tqdm(trainloader, leave=False)

    # iteration over the batches
    for batch_id, data in enumerate(t):
        # make batch tensor and target tensor
        input = data['input']
        target = data['target']
        mask = data['mask']

        if USE_CUDA:
            input = input.cuda()
            target = target.cuda()
            mask = mask.cuda()

        # initialize gradients
        optimizer.zero_grad()

        # predictions
        output = model(input)

        # mask tensor
        output = output * mask
        target = target * mask
        
        # calculate loss
        l_ = loss(output, target)
        l_ = l_ * target.size(1)
        if lamda != 0:
            reg_loss = 0
            for param in model.parameters():
                if USE_CUDA:
                    param_target = Variable(torch.zeros(param.size())).cuda()
                else:
                    param_target = Variable(torch.zeros(param.size()))
                reg_loss += l1_loss(param, param_target)

            reg_loss = lamda * reg_loss
            l_ = l_ + reg_loss

        total_loss += l_.item()
        # backward loss
        l_.backward()
        # optimizer step
        optimizer.step()

        # train conditions
        if lamda != 0:
            sys.stdout.write("\rreg_loss=%f, loss=%f" % (reg_loss.item(),l_.item()))
            sys.stdout.flush()
        else:
            sys.stdout.write("\rloss=%f" % (l_.item()))
            sys.stdout.flush()

        # visualize train conditions
        if batch_id % 30 == 0 and batch_id != 0:
            batch_loss = batch_loss + l_.item()
            batch_loss = batch_loss / 30
            # display visdom board
            phase = epoch + batch_id / epoch_size
            visualize(phase, batch_loss, win)
            batch_loss = 0
            # evaluate
            target_map = data["map"]
            model.eval()
            output = model(input)
            model_evaluate(output, target_map, v_array,
                           epoch, epoch_size, batch_id)
            model.train()
        else:
            batch_loss = batch_loss + l_.item()

    return total_loss

# Test Code

In [13]:
def model_test(testloader):
    # set model to eval mode
    model.eval()

    # define annotations
    v_array = testloader.dataset.v_array
    
    if args["ZSL"] is True:
        v_array = v_array[GT_list]
    
    v_array = torch.from_numpy(v_array)
    if USE_CUDA:
        v_array = v_array.cuda()

    # make output_dir
    if not os.path.isdir(args["output_dir"]):
        os.makedirs(args["output_dir"])

    # iteration over the batches
    for batch_id, data in enumerate(tqdm(testloader)):
        # make batch tensor and target tensor
        input = data['input']

        if USE_CUDA:
            input = input.cuda()

        # predictions
        output = model(input)

        # output segmentation_img
        filename = os.path.basename(
            testloader.dataset.get_filename(batch_id)[0])
        filename = filename.split(".")[0]
        filename = filename + ".png"
        single_output = output[0, :, :, :]
        single_output = single_output.transpose(0, 1).transpose(1, 2)
        result = min_euclidean(single_output, v_array).cpu().numpy()
        result = np.uint8(result)

        Image.fromarray(result).save(
            os.path.join(args["output_dir"], filename))

# Eval Code

In [14]:
def model_evaluate(output, target_map, v_array, epoch, epoch_size, batch_id):
    accuracy_train=[]
    accuracy_test=[]
    if args["ZSL"] is True:
        v_array2 = v_array[GT_list]
    
    for id in range(output.size(0)):
        single_output = output[id, :, :, :]
        tr_target = target_map[id, :, :].cpu().numpy()
        msk = np.isin(tr_target, GT_list)
        ctarget = tr_target.copy()
        tr_target[ctarget > 181] = 182
        single_output = single_output.transpose(0, 1).transpose(1, 2)
        tr_result = min_euclidean(single_output, v_array).cpu().numpy()
        te_result = min_euclidean(single_output, v_array2).cpu().numpy()
        te_target = tr_map_te[tr_target]
        tr_result = tr_result[~msk]==tr_target[~msk]
        te_result = te_result[msk]==te_target[msk]
        if len(tr_result) != 0:
            acc_tr = tr_result.mean()
            accuracy_train.append(acc_tr)
        if len(te_result) != 0:
            acc_te = te_result.mean()
            accuracy_test.append(acc_te)

    acc_tr = np.array(accuracy_train)
    acc_te = np.array(accuracy_test)
    acc1 = np.mean(acc_tr[~np.isnan(acc_tr)])
    acc2 = np.mean(acc_te[~np.isnan(acc_te)])
    phase = epoch + batch_id / epoch_size
    visualize(phase, acc1, win_acc)
    visualize(phase, acc2, win_tacc)

# Visualizer

In [15]:
def visualize(phase, visualized_data, window):
    vis.line(
        X=np.array([phase]),
        Y=np.array([visualized_data]),
        update='append',
        win=window
    )

# Utility

In [16]:
def min_euclidean(out, sem):
    """pytorch calculate euclidean"""
    nbr = sem.size(1)
    ab = torch.mm(out.view(-1, nbr), sem.t())
    ab = ab.view(out.size(0), out.size(1), sem.size(0))
    aa = (sem**2).sum(1)
    bb = (out**2).sum(-1)
    res = aa[None, None, :] + bb[:, :, None] - 2 * ab
    return res.min(-1)[1]

# Main

In [ ]:
# compose transforms
train_transform = transforms.Compose(
    [transforms.RandomResizedCrop(256, scale=(1.0, 1.0),ratio=(1.0, 1.0),
                                  interpolation=Image.NEAREST)]
)
test_transform = transforms.Compose(
    []
)

# initialize model
model.initialized_with_pretrained_weights()

# load model
if p_args.load is True:
    if args["model"] is True:
        model.load_from_filename(args["model_load_pth"])

# load dataset
trainset = datasets.ImageFolderDenseFileLists(
    input_root=args["input_root"], target_root=args["target_root"],
    map_root=args["map_root"], filenames=args["filenames"],
    training=True,
    model=None, config=args, transform=train_transform,
    USE_CUDA=USE_CUDA)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=args["batch_size"], shuffle=True,
    num_workers=args["batch_size"])
testset = datasets.ImageFolderDenseFileLists(
    input_root='/home/tanida/workspace/ZS_segnet/data/test/input', target_root=None,
    map_root=None, filenames='/home/tanida/workspace/ZS_segnet/data/test/names.txt',
    training=False,
    model=None, config=args, transform=test_transform,
    USE_CUDA=USE_CUDA)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=False, num_workers=1)

# train and test
for epoch in range(0, args["epochs"]):

    if p_args.test is False:
        # training
        loss = model_train(epoch, trainloader)
        print("epoch:%d,model_loss:%f" % (epoch,loss))
        # open log_file
        f_log.open()
        # write log_file
        f_log.write(epoch, loss)
        # close log_file
        f_log.close()
        # make project_dir
        if not os.path.isdir(args["project_dir"]):
            os.makedirs(args["project_dir"])
        # save checkpoint
        torch.save(model.state_dict(),
                   args["project_dir"] + "/checkpoint_" + str(epoch) +
                   ".pth")
    elif p_args.test is True and p_args.load is True:
        # test
        model_test(testloader)
        break
    else:
        print('can not test the model!')
        break
# save model
if p_args.test is False:
    torch.save(model.state_dict(), os.path.join(
        args["project_dir"], args["save_pth"]))

initialize from vgg16


loss=4.267137